# Nlp final project

1. Yaniv Gabay - 205745615 - yanivga@edu.hac.ac.il
2. Shahar Asher - 209305408 - shaharas@edu.hac.ac.il
3. Hadar Liel Harush - 211721568 - hadarhar@edu.hac.ac.il
## Eminem song generator

This project will compare different models, all trained on the legendary hip-hop artist Eminem lyrics.
We want to expirment with different models, our own model, different apis, to see and compare the results.


to be added?

In [1]:
######imports

#pip install allennlp
!pip install flair
!pip install pronouncing
import torch
import pandas as pd
from flair.data import Sentence
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings, ELMoEmbeddings,TransformerWordEmbeddings
from transformers import GPT2Model, GPT2Tokenizer
import pronouncing
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Make sure to download NLTK data (e.g., WordNet, Punkt tokenizer models)
nltk.download('wordnet')
nltk.download('punkt')

#### if you want to export data
####
debug = False



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 54.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 22.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
##### Loading the data
### we be using <startVerse> <endVerse> <startChorus><endChorus>
## to give our model some different between the verses and the choruses.

#important to export the data to see the effect
#of changes, especilly with a small dataset
def export_data(data,stage,debug_mode):
    if(not debug_mode):
        return
    try:
        data.to_csv('data_'+stage+'.csv')
    except Exception as e:
        print("Error exporting the csv file:", e)
#basic import
def import_data(location):
    df = read_data(location)
    return df
#read with try except,can add more specific
#we try different encoding, you can never know what chars you will find.
# this one is working with latin1, but we still left the function
#exceptions
def read_data(location):
    try_encodings = ['utf-8', 'latin1', 'utf-16', 'cp1252', 'ISO-8859-1', 'windows-1252']
    for encoding in try_encodings:
        try:
            df = pd.read_csv(location ,encoding=encoding,delimiter='\t')
            print(f"Success with encoding: {encoding}")
            print(df.head())
            return df
            break
        except Exception as e:
            print(f"Error with encoding {encoding}: {e}")
####
file_loc = './Eminem_Lyrics.csv'
df = import_data(file_loc)

export_data(df,'afterImport-1',debug)



Error with encoding utf-8: 'utf-8' codec can't decode byte 0x92 in position 6: invalid start byte
Success with encoding: latin1
                        Album_Name       Song_Name  \
0  Music To Be Murdered By: Side B  Alfred (Intro)   
1  Music To Be Murdered By: Side B     Black Magic   
2  Music To Be Murdered By: Side B  Alfreds Theme   
3  Music To Be Murdered By: Side B       Tone Deaf   
4  Music To Be Murdered By: Side B  Book of Rhymes   

                                              Lyrics  \
0  [Intro: Alfred Hitchcock]\nThus far, this albu...   
1  [Chorus: Skylar Grey & Eminem]\nBlack magic, n...   
2  [Verse 1]\nBefore I check the mic (Check, chec...   
3  [Intro]\nYeah, I'm sorry (Huh?)\nWhat did you ...   
4  [Intro]\nI don't smile, I don't frown, get too...   

                                           Album_URL   Views  \
0  https://genius.com/albums/Eminem/Music-to-be-m...   24.3K   
1  https://genius.com/albums/Eminem/Music-to-be-m...  180.6K   
2  https://genius.

In [3]:
#actual messing with the dataset
import re


def insert_tokens(lyrics):
    # Pattern to identify all tags (verses, choruses, etc.)
    pattern = re.compile(r'\[(Verse|Chorus|Intro|Outro)')

    # Placeholder for processed lyrics
    processed_lyrics = "<startSong>\n"
    last_tag = None

    for line in lyrics.split('\n'):
        tag_match = pattern.match(line)
        if tag_match:
            # Close the previous tag if exists
            if last_tag:
                processed_lyrics += f"<end{last_tag}>\n"
            # Update last_tag and open a new tag
            last_tag = tag_match.group(1)  # Capture the tag name (Verse or Chorus)
            processed_lyrics += f"<start{last_tag}>\n"
        else:
            processed_lyrics += line + "\n"

    # Close the last opened tag
    if last_tag:
        processed_lyrics += f"<end{last_tag}>\n"
    processed_lyrics += "<endSong>"

    return processed_lyrics

def remove_remaining_tags(lyrics):
    # Pattern to match any text within square brackets
    pattern = re.compile(r'\[[^\]]*\]')
    cleaned_lyrics = re.sub(pattern, '', lyrics)
    return cleaned_lyrics

def handle_special(lyrics):
    #remove ' and - and ? and !
    output_string = re.sub(r"['\-?!]",'', lyrics)
    return output_string

new_df = df['Lyrics'].apply(insert_tokens)

print(new_df)
export_data(new_df,'afterInsertTokens',debug)


## some songs can have some another tags, like post-chorus,pre etc.
## we arent intresnted in those, so we will just delete them in the next step
new_df = new_df.apply(remove_remaining_tags)
new_df = new_df.apply(handle_special)
#lowercase everything
new_df = new_df.str.lower()

export_data(new_df,'afterLowerAndRemove',debug)


0      <startSong>\n<startIntro>\nThus far, this albu...
1      <startSong>\n<startChorus>\nBlack magic, night...
2      <startSong>\n<startVerse>\nBefore I check the ...
3      <startSong>\n<startIntro>\nYeah, I'm sorry (Hu...
4      <startSong>\n<startIntro>\nI don't smile, I do...
                             ...                        
343    <startSong>\n<startChorus>\nI know there's som...
344    <startSong>\n<startIntro>\nYeah, yeah, I get i...
345    <startSong>\n<startVerse>\nI cut back on the s...
346    <startSong>\n<startIntro>\nC'mon!\n\n<endIntro...
347    <startSong>\n<startIntro>\nCDs ain't selling n...
Name: Lyrics, Length: 348, dtype: object


In [4]:
### special chars
def remove_non_ascii_and_print(text):
    cleaned_text = []
    removed_chars = []
    for char in text:
        if ord(char) < 128:
            cleaned_text.append(char)
        else:
            removed_chars.append(char)
    if removed_chars:
        print(f"Removed characters: {''.join(removed_chars)}")
    return ''.join(cleaned_text)
clean = new_df.apply(remove_non_ascii_and_print)


export_data(clean,'afterRemoveNonAscii',debug)

#### we still have (lyrics) in the dataset, need to decide
### we still have ! or ? in the dataset, should we remove?

Removed characters: 
Removed characters: á
Removed characters: 
Removed characters: 
Removed characters: ó
Removed characters: 
Removed characters: 
Removed characters: éé
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: ö
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: éâé
Removed characters: ó
Removed characters: 
Removed characters: áó
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: óéáí
Removed characters: 
Removed characters: ú
Removed characters: 
Removed characters: â
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters:

## we will try several tokenizer options

In [5]:
temp_switch = {'<startverse>':'startverse',
                '<endverse>':'endverse',
                '<startchorus>':'startchorus',
                '<endchorus>':'endchorus',
                '<startoutro>':'startoutro',
                '<endoutro>':'endoutro',
                '<startintro>':'startintro',
                '<endintro>':'endintro',
                '<startsong>':'startsong',
                '<endsong>':'endsong'}
def replace_tags(text, temp_switch, reverse=False):
    if reverse:
        temp_switch = {v: k for k, v in temp_switch.items()}

    for tag, replacement in temp_switch.items():
        text = text.replace(tag, replacement)
    return text

temp_clean = clean.apply(lambda x: replace_tags(x, temp_switch, reverse=False))
export_data(temp_clean,'temp_clean',debug)
!pip install spacy
import spacy
!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
spacy_tokens_df = temp_clean.apply(lambda x: [token.text for token in nlp(x)])


import nltk
from nltk.tokenize import word_tokenize
#nltk.download('punkt')
#nltk_tokens_df = temp_clean.apply(word_tokenize)


from transformers import BertTokenizer
# For BERT
tokenizer_bert = BertTokenizer.from_pretrained("bert-base-uncased")
num_of_added_tokens = tokenizer_bert.add_special_tokens({'additional_special_tokens':
                                    ['startverse', 'endverse',
                                      'startchorus', 'endchorus',
                                        'startoutro', 'endoutro',
                                        'startintro', 'endintro',
                                        'startsong', 'endsong']})
print(f"Number of added tokens: {num_of_added_tokens}")
#BERT_tokens_df = temp_clean.apply(lambda x: tokenizer_bert.tokenize(x))
from transformers import GPT2Tokenizer
# For GPT-2

tokenizer_gpt2 = GPT2Tokenizer.from_pretrained("gpt2")
num_of_added_tokens = tokenizer_gpt2.add_special_tokens({'additional_special_tokens':
                                    ['startverse', 'endverse',
                                      'startchorus', 'endchorus',
                                        'startoutro', 'endoutro',
                                        'startintro', 'endintro',
                                          'startsong', 'endsong']})
#gpt2_tokens_df = temp_clean.apply(lambda x: tokenizer_gpt2.tokenize(x))

#export_data(spacy_tokens_df,'spacy',debug)
#export_data(nltk_tokens_df,'nltk',debug)
#export_data(BERT_tokens_df,'BERT',debug)
#export_data(gpt2_tokens_df,'GPT2', debug )

#total_token_models = [spacy_tokens_df, nltk_tokens_df, BERT_tokens_df, gpt2_tokens_df]
#for model in total_token_models:
  #  print("")
   # print(model.head())

## pronouncing library
## we will use it to get the phonetic features of the words


import pronouncing

def get_phonetic_features(word):
    phones = pronouncing.phones_for_word(word)
    stress = [pronouncing.stresses(p) for p in phones] if phones else ''
    return {'phones': phones, 'stress': stress}







#spacy_phonetics_df = spacy_tokens_df.apply(lambda tokens: [get_phonetic_features(token) for token in tokens])
#nltk_phonetics_df = nltk_tokens_df.apply(lambda tokens: [get_phonetic_features(token) for token in tokens])
#bert_phonetics_df = BERT_tokens_df.apply(lambda tokens: [get_phonetic_features(token) for token in tokens])
#gpt2_phonetics_df = gpt2_tokens_df.apply(lambda tokens: [get_phonetic_features(token) for token in tokens])

#export_data(spacy_phonetics_df, 'spacy_phonetics',debug)
#export_data(nltk_phonetics_df, 'nltk_phonetics',  debug)
#xport_data(bert_phonetics_df, 'bert_phonetics', debug)
#xport_data(gpt2_phonetics_df, 'gpt2_phonetics', debug)

#total_phonetic_models = [spacy_phonetics_df, nltk_phonetics_df, bert_phonetics_df, gpt2_phonetics_df]


##### now , we need to perpare the data for the embeddings
### so padding, indexing unique tokens, etc.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 44.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [6]:
from flair.data import Sentence
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import torch

# Define stacked embeddings
stacked_embeddings = StackedEmbeddings([
    WordEmbeddings('glove'),
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
])

# Initialize DocumentPoolEmbeddings with stacked embeddings
pool_embeddings = DocumentPoolEmbeddings([stacked_embeddings])



2024-03-14 13:04:05,084 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmprtb00m5y


100%|██████████| 153M/153M [00:04<00:00, 32.9MB/s]

2024-03-14 13:04:10,222 copying /tmp/tmprtb00m5y to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2024-03-14 13:04:13,732 removing temp file /tmp/tmprtb00m5y
2024-03-14 13:04:14,059 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmpugphwckz


100%|██████████| 20.5M/20.5M [00:00<00:00, 33.0MB/s]

...

2024-03-14 13:04:26,649 copying /tmp/tmpsif51goz to cache at /root/.flair/embeddings/news-forward-0.4.1.pt


2024-03-14 13:04:26,827 removing temp file /tmp/tmpsif51goz
2024-03-14 13:04:28,755 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-backward-0.4.1.pt not found in cache, downloading to /tmp/tmpoasugbbr


100%|██████████| 69.7M/69.7M [00:02<00:00, 30.3MB/s]

2024-03-14 13:04:31,365 copying /tmp/tmpoasugbbr to cache at /root/.flair/embeddings/news-backward-0.4.1.pt


2024-03-14 13:04:31,695 removing temp file /tmp/tmpoasugbbr


In [7]:
def build_vocab(series):
    unique_tokens = set()
    for song in series:
        unique_tokens.update(song)

    # Reserve 0 for padding
    token_to_idx = {'<PAD>': 0}

    # Start indexing from 1 for the rest of the tokens
    for idx, token in enumerate(unique_tokens, start=1):
        token_to_idx[token] = idx

    # Optionally add specific indices for special tokens if not already included
    # This is just an example; adjust according to your specific tokens
   # special_tokens = ['<START>', '<END>', '<STARTVERSE>', '<ENDVERSE>']
   # for token in special_tokens:
    #    if token not in token_to_idx:
     #       token_to_idx[token] = len(token_to_idx)
    #
    return token_to_idx




def tokens_to_indices(tokenized_song, token_to_idx):
    return [token_to_idx.get(token, token_to_idx['<PAD>']) for token in tokenized_song]







import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

class TokenSongsDataset(Dataset):
    def __init__(self, series, seq_length):
        self.series = series
        self.seq_length = seq_length
        self.tokens = self.prepare_tokens()

    def prepare_tokens(self):
        all_tokens = []
        for token_list in self.series:
            all_tokens.extend(token_list)
        return all_tokens

    def __getitem__(self, idx):
        return (
            torch.tensor(self.tokens[idx:idx + self.seq_length], dtype=torch.long),
            torch.tensor(self.tokens[idx + self.seq_length], dtype=torch.long)
        )

    def __len__(self):
        return len(self.tokens) - self.seq_length




In [13]:
import torch.nn as nn
class BiLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.dropout = nn.Dropout(p=0.5)  # Adjust the dropout rate as needed
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=2, dropout=0.5, bidirectional=True)  # Add dropout between LSTM layers in multi-layer LSTM
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, sequence):
        embedded = self.embedding(sequence)
        lstm_out, _ = self.lstm(embedded)
        output = self.fc(lstm_out)
        return output





def get_vocab_size_from_series(tokenized_songs_series):
    vocabulary = set()  # Initialize an empty set to store your unique tokens

    # Iterate through each item in the Series
    for tokenized_song in tokenized_songs_series:
        # Update the vocabulary set with tokens from this song
        vocabulary.update(tokenized_song)

    # The vocabulary size is the number of unique tokens in the set
    vocab_size = len(vocabulary)
    return vocab_size

# Assuming 'spacy_tokens_series' is your Series with tokenized songs
token_to_idx = build_vocab(spacy_tokens_df)
# Apply the conversion to the entire series
indexed_tokens_series = spacy_tokens_df.apply(lambda song: tokens_to_indices(song, token_to_idx))


seq_length = 100  # Define the sequence length
dataset = TokenSongsDataset(indexed_tokens_series, seq_length)

# Vocabulary size is the size of the token_to_idx mapping
vocab_size = len(token_to_idx)

# Initialize DataLoader
batch_size = 12
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model initialization
embedding_dim = 100
hidden_dim = 256
model = BiLSTM(vocab_size, embedding_dim, hidden_dim)

# Training loop remains the same



In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for batch in dataloader:
    inputs, targets = batch  # Unpack the batch into inputs and targets
    print("Inputs shape:", inputs.shape)
    print("Targets shape:", targets.shape)
    break  # Just to check the first batch and then break the loop

# Device configuration - Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model.to(device)

num_of_epochs = 10
# Assuming you have an idx_to_token mapping from indices back to tokens
idx_to_token = {idx: token for token, idx in token_to_idx.items()}
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)  # LR scheduler
for epoch in range(num_of_epochs):
    model.train()  # Ensure the model is in training mode
    for i, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        last_outputs = outputs[:, -1, :]  # Get the predictions for the last token in each sequence
        loss = criterion(last_outputs, targets)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)  # Apply gradient clipping
        optimizer.step()

        if i % 100 == 0:  # Print every 100 batches
            # Convert inputs, outputs, and targets back to tokens
            input_tokens = [' '.join([idx_to_token.get(idx.item()) for idx in sequence]) for sequence in inputs]
            target_tokens = [idx_to_token.get(idx.item()) for idx in targets]
            output_indices = last_outputs.argmax(dim=1)  # Get the predicted token indices
            output_tokens = [idx_to_token.get(idx.item()) for idx in output_indices]

            print(f"Epoch [{epoch+1}/{num_of_epochs}], Batch [{i}], Loss: {loss.item():.4f}")
            #print("Input Sequence: ", input_tokens[0])  # Print the first sequence of the batch as an example
            print("Target Token: ", target_tokens[0])  # Print the corresponding target token
            print("Predicted Token: ", output_tokens[0])  # Print the predicted token
            print("-" * 50)

    scheduler.step()  # Adjust learning rate after each epoch




Inputs shape: torch.Size([12, 100])
Targets shape: torch.Size([12])
Using device: cuda
Epoch [1/10], Batch [0], Loss: 5.8860
Target Token:  got
Predicted Token:  

--------------------------------------------------
Epoch [1/10], Batch [100], Loss: 4.6474
Target Token:  my
Predicted Token:  i
--------------------------------------------------
Epoch [1/10], Batch [200], Loss: 4.5510
Target Token:  

Predicted Token:  

--------------------------------------------------
Epoch [1/10], Batch [300], Loss: 4.3425
Target Token:  ,
Predicted Token:  

--------------------------------------------------
Epoch [1/10], Batch [400], Loss: 5.5882
Target Token:  women
Predicted Token:  re
--------------------------------------------------
Epoch [1/10], Batch [500], Loss: 5.5498
Target Token:  what
Predicted Token:  i
--------------------------------------------------
Epoch [1/10], Batch [600], Loss: 4.4741
Target Token:  bling
Predicted Token:  ,
--------------------------------------------------
Epoc

KeyboardInterrupt: 

In [11]:
def generate_song(model, start_sequence, token_to_idx, idx_to_token, max_length=100):
    model.eval()  # Evaluation mode
    with torch.no_grad():  # No need to track gradients
        # Convert start sequence to indices
        tokens = [token_to_idx.get(token, token_to_idx['<PAD>']) for token in start_sequence]

        # Generate tokens until max_length or end token is reached
        for _ in range(max_length):
            sequence = torch.tensor(tokens[-seq_length:], dtype=torch.long).unsqueeze(0).to(device)
            output = model(sequence)
            last_output = output[0, -1, :]
            predicted_token_idx = last_output.argmax().item()  # Choose the token with the highest probability
            tokens.append(predicted_token_idx)

            # Break if the end of song token is generated
            if predicted_token_idx == token_to_idx.get('<ENDSONG>', -1):  # Assuming <ENDSONG> is your end token
                break

        # Convert token indices back to words
        generated_sequence = [idx_to_token[idx] for idx in tokens]

    return generated_sequence
idx_to_token = {idx: token for token, idx in token_to_idx.items()}
start_sequence = ['startsong']  # Adjust based on your tokens
generated_song = generate_song(model, start_sequence, token_to_idx, idx_to_token, max_length=300)  # max_length is adjustable

# Join the generated sequence into a string
generated_song_text = ' '.join(generated_song)
print(generated_song_text)


startsong 
 i m a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fuck you re a fu